In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
path = '/content/gdrive/My Drive/merged_traindf.xlsx'
df = pd.read_excel(path)
df.head()

,Unnamed: 0,AGREEMENTID,CUSTOMERID,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,FORECLOSURE,CUST_CONSTTYPE_ID,AGE,SEX,MARITAL_STATUS,QUALIFICATION,NO_OF_DEPENDENT,NETTAKEHOMEINCOME
0,0,11220001,12001000.0,11710107.24,2010-08-30,13.73,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7516698.51,7332154.12,184544.4,29042.46,29042.46,0.0,184544.4,2262176.82,9447930.42,2617476.02,9338009.12,2013-11-01,83559.0,40.06,39,19.0,0.0,0.6,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
1,1,11220001,12001000.0,11710107.24,2010-08-30,13.73,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7701242.91,7516698.51,184544.4,29042.46,29042.46,0.0,184544.4,2157466.66,9552640.58,2638212.64,9233298.96,2013-12-01,83559.0,40.06,40,18.0,0.0,0.6,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
2,2,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7885787.31,7701242.91,184544.4,29042.46,29042.46,0.0,184544.4,2051796.66,9658310.58,2657989.42,9127628.96,2014-01-01,83559.0,40.06,41,17.0,0.0,0.6,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
3,3,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7885787.31,7885787.31,0.0,29042.46,29042.46,0.0,0.0,1944730.57,9765376.67,2676370.10,9020562.87,2014-02-02,83559.0,40.06,41,17.0,0.0,0.6,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
4,4,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,8254876.10,8070331.70,184544.4,213586.86,213586.86,0.0,184544.4,1836705.34,9873401.90,2693791.64,8912537.64,2014-03-01,83559.0,40.06,43,15.0,0.0,0.6,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22


In [ ]:
# removing the unnamed: column
df.drop(df.columns[0],axis=1,inplace=True)

In [ ]:
# sorting the training data with respect to agreement id and Last receipt date
df.sort_values(by=['AGREEMENTID','LAST_RECEIPT_DATE'],ascending=[True,False],inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df.head(10)

,AGREEMENTID,CUSTOMERID,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,FORECLOSURE,CUST_CONSTTYPE_ID,AGE,SEX,MARITAL_STATUS,QUALIFICATION,NO_OF_DEPENDENT,NETTAKEHOMEINCOME
0,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,45.0,120,MUMBAI,7994.27,7994.27,0.0,8614898.28,8439420.50,175477.78,213586.86,213586.86,0.0,175477.78,116447.23,11593660.01,2725722.88,8693575.33,2014-05-05,1000000.0,40.06,45,0.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
1,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,8439420.50,8254876.10,184544.40,213586.86,213586.86,0.0,184544.40,1727712.37,9982394.87,2710245.45,8803544.67,2014-04-01,83559.0,40.06,44,14.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
2,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,8254876.10,8070331.70,184544.40,213586.86,213586.86,0.0,184544.40,1836705.34,9873401.90,2693791.64,8912537.64,2014-03-01,83559.0,40.06,43,15.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
3,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7885787.31,7885787.31,0.00,29042.46,29042.46,0.0,0.00,1944730.57,9765376.67,2676370.10,9020562.87,2014-02-02,83559.0,40.06,41,17.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
4,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7885787.31,7701242.91,184544.40,29042.46,29042.46,0.0,184544.40,2051796.66,9658310.58,2657989.42,9127628.96,2014-01-01,83559.0,40.06,41,17.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
5,11220001,12001000.0,11710107.24,2010-08-30,13.73,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7701242.91,7516698.51,184544.40,29042.46,29042.46,0.0,184544.40,2157466.66,9552640.58,2638212.64,9233298.96,2013-12-01,83559.0,40.06,40,18.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
6,11220001,12001000.0,11710107.24,2010-08-30,13.73,10.61,58.0,120,MUMBAI,7994.27,7994.27,0.0,7516698.51,7332154.12,184544.40,29042.46,29042.46,0.0,184544.40,2262176.82,9447930.42,2617476.02,9338009.12,2013-11-01,83559.0,40.06,39,19.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22
7,11220002,12001001.0,19290253.32,2010-09-15,13.73,10.61,137.0,180,MUMBAI,100504.58,100504.58,0.0,10619027.21,10339579.13,279448.08,0.00,0.00,0.0,279448.08,12347597.30,6942656.03,4945153.59,17138606.02,2013-11-01,126530.0,84.31,38,99.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,44.0,M,M,POSTGRAD,0.0,242703.98
8,11220006,12001005.0,3933395.00,2010-11-01,15.61,11.55,312.0,180,MUMBAI,5947.63,5947.63,0.0,4670210.68,4670210.68,0.00,1179715.72,1179715.72,0.0,0.00,2994183.52,939211.48,2990124.01,3744831.42,2017-08-05,22878.0,50.89,81,231.0,0.0,0.72,HL,10901101.0,NaN,NaN,1,3.0,43.0,M,M,GRAD,0.0,53536.21
9,11220006,12001005.0,3933395.00,2010-11-01,15.61,11.55,312.0,180,MUMBAI,5947.63,5947.63,0.0,4619683.43,4619683.43,0.00,1179715.72,1179715.72,0.0,0.00,2997308.21,936086.79,2958902.06,3747956.11,2017-07-05,22878.0,50.89,80,232.0,0.0,0.72,HL,10901101.0,NaN,NaN,1,3.0,43.0,M,M,GRAD,0.0,53536.21


The dataframe is sorted in such a way that the all the agreement ids' are presented sequentially and loan repayment record for the respective agreement ids' is displayed starting with the latest record of the payment of monthly installment first and then going backwards in time,in the next rows.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374585 entries, 0 to 374584
Data columns (total 42 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   AGREEMENTID            374585 non-null  int64         
 1   CUSTOMERID             374272 non-null  float64       
 2   NET_DISBURSED_AMT      374585 non-null  float64       
 3   INTEREST_START_DATE    374585 non-null  datetime64[ns]
 4   CURRENT_ROI            374585 non-null  float64       
 5   ORIGNAL_ROI            374585 non-null  float64       
 6   CURRENT_TENOR          374582 non-null  float64       
 7   ORIGNAL_TENOR          374585 non-null  int64         
 8   CITY                   374577 non-null  object        
 9   PRE_EMI_DUEAMT         374585 non-null  float64       
 10  PRE_EMI_RECEIVED_AMT   374585 non-null  float64       
 11  PRE_EMI_OS_AMOUNT      374585 non-null  float64       
 12  EMI_DUEAMT             374585 non-null  floa

As it can be observed from the dataframe info() summary, there is a disparity in the no. of agreement ids and customer id,which suggests that some of the loan agreement ids are not assigned to any particular customer. Such agreeement ids are deeemed of no use in the process of model building because they wont have any record associated with it,hence those rows are dropped.

In [ ]:
df.dropna(subset=['CUSTOMERID'],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374272 entries, 0 to 374584
Data columns (total 42 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   AGREEMENTID            374272 non-null  int64         
 1   CUSTOMERID             374272 non-null  float64       
 2   NET_DISBURSED_AMT      374272 non-null  float64       
 3   INTEREST_START_DATE    374272 non-null  datetime64[ns]
 4   CURRENT_ROI            374272 non-null  float64       
 5   ORIGNAL_ROI            374272 non-null  float64       
 6   CURRENT_TENOR          374269 non-null  float64       
 7   ORIGNAL_TENOR          374272 non-null  int64         
 8   CITY                   374264 non-null  object        
 9   PRE_EMI_DUEAMT         374272 non-null  float64       
 10  PRE_EMI_RECEIVED_AMT   374272 non-null  float64       
 11  PRE_EMI_OS_AMOUNT      374272 non-null  float64       
 12  EMI_DUEAMT             374272 non-null  floa

In [ ]:
print(df.AGREEMENTID.nunique())
print(df.CUSTOMERID.nunique())

19731
19731


## Feature Engineering

In an attempt to solve this problem of predicting whether a particular loan, based on its historical loan repayment record,would foreclose or not,we create few lag features based on available features to convert the tabular data from a long format to a wide format.The past 6 month history for each of the loan would be considered to train the model. 

lag features were created for the following predictors


*   PAID PRINCIPAL -  It would show the cumulative repayment of the principal amount
*   PAID INTEREST - It would show the cumulative repayment of the interest on the loan amount
*   OUTSTANDING EMI - the amount in monthly EMI installment which was not received
*   DAYS PAST DUE- the no. of days elapsed since the date on which previous month's installment was meant to be paid. 





In [ ]:
# lag features for paid principal
for i in range(-1,-7,-1):
    t = df.groupby(['AGREEMENTID']).PAID_PRINCIPAL.shift(i)
    coln = 'PaidPrn'+str(i)
    df[coln] = t

In [ ]:
# lag features for paid interest
for i in range(-1,-7,-1):
    t = df.groupby(['AGREEMENTID']).PAID_INTEREST.shift(i)
    coln = 'PaidIntrst'+str(i)
    df[coln] = t

In [ ]:
# lag features for outstanding EMI
for i in range(-1,-7,-1):
    t = df.groupby(['AGREEMENTID']).EMI_OS_AMOUNT.shift(i)
    coln = 'EmiDue'+str(i)
    df[coln] = t

In [ ]:
# lag features for DPD
for i in range(-1,-7,-1):
    t = df.groupby(['AGREEMENTID']).DPD.shift(i)
    coln = 'DPD'+str(i)
    df[coln] = t

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374272 entries, 0 to 374584
Data columns (total 66 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   AGREEMENTID            374272 non-null  int64         
 1   CUSTOMERID             374272 non-null  float64       
 2   NET_DISBURSED_AMT      374272 non-null  float64       
 3   INTEREST_START_DATE    374272 non-null  datetime64[ns]
 4   CURRENT_ROI            374272 non-null  float64       
 5   ORIGNAL_ROI            374272 non-null  float64       
 6   CURRENT_TENOR          374269 non-null  float64       
 7   ORIGNAL_TENOR          374272 non-null  int64         
 8   CITY                   374264 non-null  object        
 9   PRE_EMI_DUEAMT         374272 non-null  float64       
 10  PRE_EMI_RECEIVED_AMT   374272 non-null  float64       
 11  PRE_EMI_OS_AMOUNT      374272 non-null  float64       
 12  EMI_DUEAMT             374272 non-null  floa

Now, The dataframe is collapsed by removing duplicates,which means every row in the dataframe now has the loan repayment record for each of the unique loans. Every row is the last transaction detail in the history of the respective loans,along with the lag features of each loan, appended at the end.

In [ ]:
cols = ['AGREEMENTID','CUSTOMERID']

In [ ]:
df.drop_duplicates(cols,inplace=True)

In [ ]:
df.head(10)

,AGREEMENTID,CUSTOMERID,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,FORECLOSURE,CUST_CONSTTYPE_ID,AGE,SEX,MARITAL_STATUS,QUALIFICATION,NO_OF_DEPENDENT,NETTAKEHOMEINCOME,PaidPrn-1,PaidPrn-2,PaidPrn-3,PaidPrn-4,PaidPrn-5,PaidPrn-6,PaidIntrst-1,PaidIntrst-2,PaidIntrst-3,PaidIntrst-4,PaidIntrst-5,PaidIntrst-6,EmiDue-1,EmiDue-2,EmiDue-3,EmiDue-4,EmiDue-5,EmiDue-6,DPD-1,DPD-2,DPD-3,DPD-4,DPD-5,DPD-6
0,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,45.0,120,MUMBAI,7994.27,7994.27,0.0,8614898.28,8439420.50,175477.78,213586.86,213586.86,0.00,175477.78,116447.23,11593660.01,2725722.88,8693575.33,2014-05-05,1000000.0,40.06,45,0.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22,9982394.87,9873401.90,9765376.67,9658310.58,9552640.58,9447930.42,2710245.45,2693791.64,2676370.10,2657989.42,2638212.64,2617476.02,184544.40,184544.40,0.00,184544.40,184544.40,184544.40,0.0,0.0,0.0,0.0,0.0,0.0
7,11220002,12001001.0,19290253.32,2010-09-15,13.73,10.61,137.0,180,MUMBAI,100504.58,100504.58,0.0,10619027.21,10339579.13,279448.08,0.00,0.00,0.00,279448.08,12347597.30,6942656.03,4945153.59,17138606.02,2013-11-01,126530.0,84.31,38,99.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,44.0,M,M,POSTGRAD,0.0,242703.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11220006,12001005.0,3933395.00,2010-11-01,15.61,11.55,312.0,180,MUMBAI,5947.63,5947.63,0.0,4670210.68,4670210.68,0.00,1179715.72,1179715.72,0.00,0.00,2994183.52,939211.48,2990124.01,3744831.42,2017-08-05,22878.0,50.89,81,231.0,0.0,0.72,HL,10901101.0,NaN,NaN,1,3.0,43.0,M,M,GRAD,0.0,53536.21,936086.79,932994.31,929933.73,926904.68,924444.55,922093.00,2958902.06,2927647.89,2896361.83,2865044.23,2833157.71,2801162.62,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
54,11220008,12001007.0,10022587.71,2010-10-06,11.24,10.61,91.0,180,THANE,60322.18,60322.18,0.0,13130976.81,13067317.53,63659.28,6770848.42,6770848.42,0.00,63659.28,42951.20,9979636.50,4648994.27,5747875.24,2018-05-02,65741.0,84.63,91,0.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,123370.48,9881994.27,9785078.90,9688884.98,9593407.14,9498640.07,9404578.45,4647939.82,4646158.51,4643655.74,4640436.89,4636507.28,4631872.21,145192.42,145192.42,145192.42,145192.42,145192.42,145192.42,0.0,0.0,0.0,0.0,0.0,0.0
109,11220010,12001009.0,7755937.31,2010-10-26,14.98,10.92,304.0,180,MUMBAI,27732.79,27732.79,0.0,10489232.82,10489232.82,0.00,120218.11,120218.11,0.00,0.00,7200652.61,555284.70,6593777.81,7200652.61,2018-04-05,54433.0,30.94,89,215.0,0.0,0.83,HL,10901101.0,NaN,NaN,1,3.0,37.0,M,M,UG,0.0,93803.84,545667.34,536145.20,526717.35,517382.83,508140.74,498990.16,6521675.14,6449477.23,6377185.05,6304799.54,6232321.60,6159752.14,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
163,11220011,12001010.0,954408.27,2010-10-28,12.42,10.61,235.0,180,THANE,1325.13,1325.13,0.0,1355118.92,1341291.17,13827.74,27655.49,27655.49,0.00,13827.74,769138.61,185269.66,727393.58,769138.61,2018-12-07,6261.0,23.24,98,137.0,0.0,0.60,HL,10901100.0,NaN,NaN,0,1.0,30.0,M,M,GRAD,0.0,35880.97,182272.34,179299.67,175868.86,172464.15,169085.35,165601.78,720991.29,714564.35,708595.54,702600.64,696579.83,690663.78,13827.74,13827.74,13827.74,13827.74,13827.74,13827.74,0.0,0.0,0.0,0.0,0.0,0.0
225,11220012,12001011.0,6605701.52,2010-11-19,16.17,10.92,390.0,300,MUMBAI,37790.53,37790.53,0.0,9101193.23,8910126.97,191066.25,95533.13,95533.13,0.00,191066.25,5765533.27,840168.25,5692699.41,6215922.01,2018-12-11,43256.0,60.69,96,294.0,56.0,0.55,HL,10901100.0,NaN,NaN,0,1.0,33.0,M,M,GRAD,0.0,63576

In [ ]:
# The distribution of target variable in training data
df.FORECLOSURE.value_counts()

0    17936
1     1795
Name: FORECLOSURE, dtype: int64

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19731 entries, 0 to 19730
Data columns (total 66 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   AGREEMENTID            19731 non-null  int64         
 1   CUSTOMERID             19731 non-null  float64       
 2   NET_DISBURSED_AMT      19731 non-null  float64       
 3   INTEREST_START_DATE    19731 non-null  datetime64[ns]
 4   CURRENT_ROI            19731 non-null  float64       
 5   ORIGNAL_ROI            19731 non-null  float64       
 6   CURRENT_TENOR          19731 non-null  float64       
 7   ORIGNAL_TENOR          19731 non-null  int64         
 8   CITY                   19731 non-null  object        
 9   PRE_EMI_DUEAMT         19731 non-null  float64       
 10  PRE_EMI_RECEIVED_AMT   19731 non-null  float64       
 11  PRE_EMI_OS_AMOUNT      19731 non-null  float64       
 12  EMI_DUEAMT             19731 non-null  float64       
 13  E

In [ ]:
print(df.NPA_IN_LAST_MONTH.unique())
print(df.NPA_IN_CURRENT_MONTH.unique())

[nan 'Yes' 0 '#N/']
[nan 'Yes' 0]


In [ ]:
df.NPA_IN_LAST_MONTH.replace({'Yes':1,'#N/':np.nan}, inplace=True)
df.NPA_IN_CURRENT_MONTH.replace({'Yes':1}, inplace=True)

In [ ]:
print(df.NPA_IN_LAST_MONTH.unique())
print(df.NPA_IN_CURRENT_MONTH.unique())

[nan  1.  0.]
[nan  1.  0.]


In [ ]:
df.head(10)

,AGREEMENTID,CUSTOMERID,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,FORECLOSURE,CUST_CONSTTYPE_ID,AGE,SEX,MARITAL_STATUS,QUALIFICATION,NO_OF_DEPENDENT,NETTAKEHOMEINCOME,PaidPrn-1,PaidPrn-2,PaidPrn-3,PaidPrn-4,PaidPrn-5,PaidPrn-6,PaidIntrst-1,PaidIntrst-2,PaidIntrst-3,PaidIntrst-4,PaidIntrst-5,PaidIntrst-6,EmiDue-1,EmiDue-2,EmiDue-3,EmiDue-4,EmiDue-5,EmiDue-6,DPD-1,DPD-2,DPD-3,DPD-4,DPD-5,DPD-6
0,11220001,12001000.0,11710107.24,2010-08-30,13.42,10.61,45.0,120,MUMBAI,7994.27,7994.27,0.0,8614898.28,8439420.50,175477.78,213586.86,213586.86,0.00,175477.78,116447.23,11593660.01,2725722.88,8693575.33,2014-05-05,1000000.0,40.06,45,0.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,198375.22,9982394.87,9873401.90,9765376.67,9658310.58,9552640.58,9447930.42,2710245.45,2693791.64,2676370.10,2657989.42,2638212.64,2617476.02,184544.40,184544.40,0.00,184544.40,184544.40,184544.40,0.0,0.0,0.0,0.0,0.0,0.0
1,11220002,12001001.0,19290253.32,2010-09-15,13.73,10.61,137.0,180,MUMBAI,100504.58,100504.58,0.0,10619027.21,10339579.13,279448.08,0.00,0.00,0.00,279448.08,12347597.30,6942656.03,4945153.59,17138606.02,2013-11-01,126530.0,84.31,38,99.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,44.0,M,M,POSTGRAD,0.0,242703.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11220006,12001005.0,3933395.00,2010-11-01,15.61,11.55,312.0,180,MUMBAI,5947.63,5947.63,0.0,4670210.68,4670210.68,0.00,1179715.72,1179715.72,0.00,0.00,2994183.52,939211.48,2990124.01,3744831.42,2017-08-05,22878.0,50.89,81,231.0,0.0,0.72,HL,10901101.0,NaN,NaN,1,3.0,43.0,M,M,GRAD,0.0,53536.21,936086.79,932994.31,929933.73,926904.68,924444.55,922093.00,2958902.06,2927647.89,2896361.83,2865044.23,2833157.71,2801162.62,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,11220008,12001007.0,10022587.71,2010-10-06,11.24,10.61,91.0,180,THANE,60322.18,60322.18,0.0,13130976.81,13067317.53,63659.28,6770848.42,6770848.42,0.00,63659.28,42951.20,9979636.50,4648994.27,5747875.24,2018-05-02,65741.0,84.63,91,0.0,0.0,0.60,HL,10901100.0,NaN,NaN,1,1.0,33.0,M,M,POSTGRAD,0.0,123370.48,9881994.27,9785078.90,9688884.98,9593407.14,9498640.07,9404578.45,4647939.82,4646158.51,4643655.74,4640436.89,4636507.28,4631872.21,145192.42,145192.42,145192.42,145192.42,145192.42,145192.42,0.0,0.0,0.0,0.0,0.0,0.0
4,11220010,12001009.0,7755937.31,2010-10-26,14.98,10.92,304.0,180,MUMBAI,27732.79,27732.79,0.0,10489232.82,10489232.82,0.00,120218.11,120218.11,0.00,0.00,7200652.61,555284.70,6593777.81,7200652.61,2018-04-05,54433.0,30.94,89,215.0,0.0,0.83,HL,10901101.0,NaN,NaN,1,3.0,37.0,M,M,UG,0.0,93803.84,545667.34,536145.20,526717.35,517382.83,508140.74,498990.16,6521675.14,6449477.23,6377185.05,6304799.54,6232321.60,6159752.14,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
5,11220011,12001010.0,954408.27,2010-10-28,12.42,10.61,235.0,180,THANE,1325.13,1325.13,0.0,1355118.92,1341291.17,13827.74,27655.49,27655.49,0.00,13827.74,769138.61,185269.66,727393.58,769138.61,2018-12-07,6261.0,23.24,98,137.0,0.0,0.60,HL,10901100.0,NaN,NaN,0,1.0,30.0,M,M,GRAD,0.0,35880.97,182272.34,179299.67,175868.86,172464.15,169085.35,165601.78,720991.29,714564.35,708595.54,702600.64,696579.83,690663.78,13827.74,13827.74,13827.74,13827.74,13827.74,13827.74,0.0,0.0,0.0,0.0,0.0,0.0
6,11220012,12001011.0,6605701.52,2010-11-19,16.17,10.92,390.0,300,MUMBAI,37790.53,37790.53,0.0,9101193.23,8910126.97,191066.25,95533.13,95533.13,0.00,191066.25,5765533.27,840168.25,5692699.41,6215922.01,2018-12-11,43256.0,60.69,96,294.0,56.0,0.55,HL,10901100.0,NaN,NaN,0,1.0,33.0,M,M,GRAD,0.0,63576.87,834

In [ ]:
# df.to_excel("cleaned_traindt.xlsx",float_format='%.2f',index=False)